<a href="https://colab.research.google.com/github/fzanart/Socialz/blob/wip/mu_lambda_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install scipy --upgrade

In [47]:
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import qmc

In [48]:
data = {}


def generate_random(number):
  values = []
  for i in range(10):
      values.extend([random.uniform(0, 1)])
  return values

#data['Betweeness'] = generate_random(10)
data['x'] = generate_random(10)
data['y'] = generate_random(10)

df = pd.DataFrame(data)
df

,x,y
0,0.035485,0.180070
1,0.441429,0.642465
2,0.258826,0.510717
3,0.508317,0.532324
4,0.609645,0.424542
5,0.900622,0.090174
6,0.090906,0.602147
7,0.093987,0.560369
8,0.311102,0.074123
9,0.437365,0.361757


mu: The number of parents selected each iteration.

lambda: Size of the population.

lambda / mu: Number of children generated from each selected parent.

In [50]:
sample = df.sample(frac=0.2, random_state=0)
sample

,x,y
2,0.258826,0.510717
8,0.311102,0.074123


In [51]:
replace = np.random.rand(2,2)
replace

array([[0.44663206, 0.0604282 ],
       [0.75950495, 0.08489605]])

In [52]:
df.loc[sample.index] = replace

In [53]:
df

,x,y
0,0.035485,0.180070
1,0.441429,0.642465
2,0.446632,0.060428
3,0.508317,0.532324
4,0.609645,0.424542
5,0.900622,0.090174
6,0.090906,0.602147
7,0.093987,0.560369
8,0.759505,0.084896
9,0.437365,0.361757


In [5]:
def objective(candidate):
  return qmc.discrepancy(candidate, method='L2-star',workers=-1)

In [54]:
objective(df)

0.13884680797639343

In [65]:
from pandas.core import indexers
# the number of children generated by parents
lam = 10

# initial population
population = list()
indexes = list()

for i in range(lam):
  candidate = df.copy()
  sample = candidate.sample(frac=0.2, random_state=i)
  indexes.append(sample.index)
  replace = np.random.rand(2,2)
  candidate.loc[sample.index] = replace
  population.append(candidate)

In [66]:
scores = [objective(candidate) for candidate in population]
scores

[0.14625853090323712,
 0.10825840223298916,
 0.16457107258245232,
 0.22235871874891674,
 0.14187057852115795,
 0.1298965697728559,
 0.1437100226433932,
 0.17994775505750674,
 0.09857980443454534,
 0.1383641288715643]

In [67]:
ranks = np.argsort(np.argsort(scores))
ranks

array([6, 1, 7, 9, 4, 2, 5, 8, 0, 3])

In [68]:
# number of parents selected
mu = 8

# select the indexes for the top mu ranked solutions
# drop the worse results	
selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
selected

[0, 1, 2, 4, 5, 6, 8, 9]

In [69]:
# create children from parents
offspring = list()

In [71]:
best, best_eval = None, 1e+10
n_children = int(lam / mu)
step_size = 0.15

In [ ]:
# TODO: function to check the offspring is between the bounds, it might be out of 0,1

for i in selected:
  if scores[i] < best_eval:
    best, best_eval = population[i], scores[i]
  for j in range(n_children):
    replace = population[i].loc[indexes[i]] + np.random.rand(2,2) * step_size
    population[i].loc[indexes[i]] = replace
  offspring.append(population[i])

In [6]:
# check if a point is within the bounds of the search
def in_bounds(point, bounds):
	# enumerate all dimensions of the point
	for d in range(len(bounds)):
		# check if out of bounds for this dimension
		if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
			return False
	return True

In [ ]:
# evolution strategy (mu, lambda) algorithm
def es_comma(objective, bounds, n_iter, step_size, mu, lam):
	best, best_eval = None, 1e+10
	# calculate the number of children per parent
	n_children = int(lam / mu)
	# initial population
	population = list()
	for _ in range(lam):
		candidate = None
		while candidate is None or not in_bounds(candidate, bounds):
			candidate = bounds[:, 0] + np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
		population.append(candidate)
	# perform the search
	for epoch in range(n_iter):
		# evaluate fitness for the population
		scores = [objective(df, c) for c in population]
		# rank scores in ascending order
		ranks = np.argsort(np.argsort(scores))
		# select the indexes for the top mu ranked solutions
		selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
		# create children from parents
		children = list()
		for i in selected:
			# check if this parent is the best solution ever seen
			if scores[i] < best_eval:
				best, best_eval = population[i], scores[i]
				print('%d, Best: (%s) = %.5f' % (epoch, best, best_eval))
			# create children for parent
			for _ in range(n_children):
				child = None
				while child is None or not in_bounds(child, bounds):
					child = population[i] + np.random.randn(len(bounds)) * step_size
				children.append(child)
		# replace population with children
		population = children
	return [best, best_eval]

In [ ]:
# define range for input
bounds = np.asarray([[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]])
# define the total iterations
n_iter = 5000
# define the maximum step size
step_size = 0.15
# number of parents selected
mu = 20
# the number of children generated by parents
lam = 100
# perform the evolution strategy (mu, lambda) search
best, score = es_comma(objective, bounds, n_iter, step_size, mu, lam)
print('(%s) = %f' % (best, score))

0, Best: f([0.55494024 0.3177519  0.18322595]) = 0.06221
0, Best: f([0.28687834 0.68565626 0.49322849]) = 0.05981
0, Best: f([0.43143179 0.89198558 0.1054242 ]) = 0.05942
0, Best: f([0.16033003 0.57356178 0.42532656]) = 0.05721
0, Best: f([0.37080323 0.43087518 0.21886505]) = 0.05408
0, Best: f([0.05058689 0.71347696 0.00414167]) = 0.05355
1, Best: f([0.10572367 0.60085753 0.17859806]) = 0.05239
1, Best: f([0.08687098 0.57745599 0.21997134]) = 0.05220
2, Best: f([0.24415256 0.58145931 0.06847236]) = 0.05183
2, Best: f([0.12677881 0.47718048 0.22297827]) = 0.05161
2, Best: f([0.12607332 0.57495417 0.08845359]) = 0.05138
2, Best: f([0.21001087 0.46489243 0.12150665]) = 0.05104
3, Best: f([0.12052145 0.49032309 0.1120397 ]) = 0.05071
52, Best: f([0.10786047 0.5084625  0.10378636]) = 0.05070
80, Best: f([0.11776317 0.49959932 0.11525603]) = 0.05069
311, Best: f([0.11586735 0.49416038 0.10857676]) = 0.05068
1332, Best: f([0.10625531 0.49730091 0.11310836]) = 0.05067
1434, Best: f([0.1082216